### First install the dependency

In [9]:
pip install redshift-connector pandas argparse

  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
Note: you may need to restart the kernel to use updated packages.


### Import all dependencies are needed

In [10]:
import csv
from zipfile import ZipFile, ZipInfo
from io import BytesIO, StringIO
from typing import IO, List
from datetime import datetime
from sys import exit, stdout
from redshift_connector import connect, Connection
import os
import argparse
import pandas as pd

In [4]:
def redshift_open_connection_by_dict(dict_secret: dict, database: str = None) -> Connection:
    print(f'Opening connection by secret dict')

    if database is None:
        database = dict_secret['dbname']

    return connect(
        host=dict_secret['hostname'],
        database=database,
        user=dict_secret['user'],
        password=dict_secret['password'],
        port=dict_secret['port']
    )

In [5]:
# Function to save data in chunks to CSV
def redshift_get_rows_and_save_csv_by_chunks(dict_secret: dict, str_query: str, save_path: str, delimiter: str = '|', quoting: int = csv.QUOTE_NONNUMERIC, lineterminator: str = '\r\n', upper_header: bool = True, batch_size: int = 1000):
    conn = redshift_open_connection_by_dict(dict_secret)
    conn.autocommit = False
    cur = conn.cursor()

    print('Executing query...')
    cur.execute(str_query)
    cols = [a[0] for a in cur.description]

    # Ensure the directory exists
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    with open(save_path, mode='w', newline='', encoding='utf-8') as csv_file:
        writer = csv.writer(csv_file, delimiter=delimiter, quotechar='"', quoting=quoting, lineterminator=lineterminator)

        if upper_header:
            cols = [k.upper() for k in cols]
            
        # Write header
        writer.writerow(cols)
        idx = 1
        while True:
            print(f'Running idx batch_size: {idx}')
            idx += 1
            rows = cur.fetchmany(batch_size)
            if not rows:
                break
            writer.writerows(rows)

    print(f"Data saved to CSV at: {save_path}")

    cur.close()
    conn.close()

In [11]:
def redshift_get_rows_and_return_dataframe_by_chunks(dict_secret: dict, str_query: str, upper_header: bool = True, batch_size: int = 1000):
    conn = redshift_open_connection_by_dict(dict_secret)
    conn.autocommit = False
    cur = conn.cursor()

    print('Executing query...')
    cur.execute(str_query)
    cols = [a[0] for a in cur.description]

    if upper_header:
        cols = [k.upper() for k in cols]

    idx = 1
    dataframes = []

    while True:
        print(f'Fetching batch number: {idx}')
        idx += 1
        rows = cur.fetchmany(batch_size)
        if not rows:
            break
        df_batch = pd.DataFrame(rows, columns=cols)
        dataframes.append(df_batch)

    print("Data fetched and stored in DataFrame.")

    cur.close()
    conn.close()

    # Concatenate all dataframes into a single DataFrame
    df_result = pd.concat(dataframes, ignore_index=True)
    return df_result

## Set your credencials and root path to store files

In [6]:
dict_secret = {
    "dbname": "datalake_dw",
    "port": 5439,
    "hostname": "asgard-redshift-production.cmqegk5gj3mi.sa-east-1.redshift.amazonaws.com",
    "user": "herculano_cunha",
    "password": ""
}
str_path_save = "/home/jovyan/work/files"


In [7]:
list_of_querys = [
    {'query': "select * from credit_portfolio.metrica limit 100000", 'filename': 'metrica.csv'},
    {'query': "select * from credit_portfolio.contrato limit 100000", 'filename': 'contrato.csv'}
]

In [ ]:
for dict_query in list_of_querys:
    print(f"Selecting rows on Redshift for query: {dict_query.get('query')}")
    save_path = f"{str_path_save}/{dict_query.get('filename')}"
    redshift_get_rows_and_save_csv_by_chunks(dict_secret=dict_secret, str_query=dict_query.get('query'), save_path=save_path, delimiter= '|', quoting=csv.QUOTE_NONE)
    print(f"{save_path} - saved")
    print()
    print('------------------------')
    print()

In [12]:
redshift_get_rows_and_return_dataframe_by_chunks(dict_secret=dict_secret, str_query="select * from credit_portfolio.metrica limit 100000")

Opening connection by secret dict
Executing query...
Fetching batch number: 1
Fetching batch number: 2
Fetching batch number: 3
Fetching batch number: 4
Fetching batch number: 5
Fetching batch number: 6
Fetching batch number: 7
Fetching batch number: 8
Fetching batch number: 9
Fetching batch number: 10
Fetching batch number: 11
Fetching batch number: 12
Fetching batch number: 13
Fetching batch number: 14
Fetching batch number: 15
Fetching batch number: 16
Fetching batch number: 17
Fetching batch number: 18
Fetching batch number: 19
Fetching batch number: 20
Fetching batch number: 21
Fetching batch number: 22
Fetching batch number: 23
Fetching batch number: 24
Fetching batch number: 25
Fetching batch number: 26
Fetching batch number: 27
Fetching batch number: 28
Fetching batch number: 29
Fetching batch number: 30
Fetching batch number: 31
Fetching batch number: 32
Fetching batch number: 33
Fetching batch number: 34
Fetching batch number: 35
Fetching batch number: 36
Fetching batch numbe

,MTR_FK_CONTRATO,MTR_FK_PARCELA,MTR_VL_PRESENTE,MTR_VL_NOMINAL_ATUAL,MTR_NR_CONTRATO_PRAZO,MTR_NR_PARCELA_PRAZO,MTR_NR_DEVEDOR_PRAZO,MTR_TX_PDD_FAIXA,MTR_VL_PDD,MTR_VL_ATRASO_SALDO,MTR_DD_METRICA,MTR_VL_JUROS_MORATORIO,MTR_VL_MULTA_MORATORIA,MTR_NR_DIA_UTIL,MTR_DT_PROCESSAMENTO
0,17030,75109,538.92,594.68,7,38,7,0,14.12,0.00,2019-06-28,0.00,0.00,26,None
1,17033,75118,36.89,42.96,5,38,5,0,0.97,0.00,2019-06-28,0.00,0.00,26,None
2,17035,75129,116.71,118.94,7,7,7,0,3.06,0.00,2019-06-28,0.00,0.00,5,None
3,17035,75130,107.79,118.94,7,38,7,0,2.82,0.00,2019-06-28,0.00,0.00,26,None
4,17049,75179,179.09,182.00,5,5,5,0,4.69,0.00,2019-06-28,0.00,0.00,3,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,133139,445107,18.67,19.40,15,15,15,0,0.49,0.00,2019-07-31,0.00,0.00,11,None
99996,160076,536582,28.01,29.48,12,12,12,0,0.73,0.00,2019-07-31,0.00,0.00,8,None
99997,160083,536609,64.61,79.47,-16,76,-16,1,10.18,0.00,2019-07-31,0.00,0.00,54,None
99998,160093,536633,95.11,95.11,-2,-2,-2,1,14.99,95.11,2019-07-31,0.13,1.34,-2,None
